# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
#импортируем библиотеки, которые будут использованы в данном исследовании и изучим общую информацию о файле
import pandas as pd

from pymystem3 import Mystem
m = Mystem()

data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


###### Описание представленных в таблице данных
- `children` — количество детей в семье;
- `days_employed` — общий трудовой стаж в днях;
- `dob_years` — возраст клиента в годах;
- `education` — уровень образования клиента;
- `education_id` — идентификатор уровня образования;
- `family_status` — семейное положение;
- `family_status_id` — идентификатор семейного положения;
- `gender` — пол клиента;
- `income_type` — тип занятости;
- `debt` — имел ли задолженность по возврату кредитов;
- `total_income` — ежемесячный доход;
- `purpose` — цель получения кредита.

In [2]:
#посмотрим первые 10 значений датасета
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#так как столбец 'children' потребуется нам для вывода, посмотрим категории, перечисленные в данном столбце
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце 'children' есть 76 клиентов с 20 детьми, что маловероятно и, скорее всего, является ошибкой ввода, а также 47 клиентов с минусовым количеством детей, что тоже является ошибкой.

In [4]:
#также посчитаем категории в столбце 'family_status'
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В этом столбце есть значения, записанные в разном регистре и их нужно привести к одному регистру.

**Вывод**

1) Наименования столбцов в датасете правильные, переименовывать их не нужно.

2) В столбцах есть значения со знаком "минус", чего не может быть и значит эти значения нужно перевести в значения без минуса. 

3) В столбцах с образованием и семейным положением есть значения, записанные в разных регистрах и все их нужно привести к нижнему регистру.

4) Пропуски есть только в столбцах с трудовым стажем в днях и в ежемесячном доходе. Так как цели исследования разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, а также есть ли зависимость между уровнем дохода и возвратом кредита в срок, то пропущенные значения в столбце с трудовым стажем не окажут влияния на результаты исследования и их можно заменить нулями, а вот уровень дохода нужно будет заполнить медианными значениями в зависимости от типа занятости клиента.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
#посмотрим на количество пропусков в датасете.
display(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Из информации о файле мы видим, что пропущено более 10 % значений и потеря этих данных может сильно сказаться на результатах исследования, которое направлено в первую очередь на соотношение семейного положения и количества детей клиента на факт погашения кредита в срок.

In [6]:
#посмотрим первые 5 пропущенных значений
display(data[data['days_employed'].isna()].head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Пропущены значения в двух разных столбцах в одних и тех же строках и вероятнее всего, это не техническая ошибка, а информация не была внесена сотрудниками банка, либо некорректно выгружена.

In [7]:
#заменим информацию о доходе медианными значениями в зависимости от типа занятости, так как нулевые значения испортят статистику
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

In [8]:
#заменим данные о количестве дней работы нулями, т.к. этот столбец нам не требуется для анализа
data['days_employed'] = data['days_employed'].fillna(0)

In [9]:
#проверим датасет на пропуски
display(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски в обеих колонках заполнены.

### Замена типа данных

Здесь мы приведем данные в столбцах 'education' и 'family_status' к нижнему регистру, заменим дробные числа float64 на целочисленные int64 в столбцах 'days_employed' и 'total_income', а также заменим в столбце 'children' отрицательные значения на положительные.

In [10]:
#приведем все данные к нижнему регистру
data['education']=data['education'].str.lower()
data['family_status']=data['family_status'].str.lower()

In [11]:
#проверим, что изменение произошло
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [12]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [13]:
#переведем количество дней работы в целое число, т.к. дни работы могут быть только целыми числами
#столбцы с доходом тоже переведем в целые числа для удобства, копейки в исследовании большой роли не сыграют
#для замены используем метод astype()
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')

In [14]:
#проверим, что преобразование прошло успешно
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [15]:
#произведем замену количества дней работы с отрицательного на положительное
data['days_employed']=abs(data['days_employed'])

In [16]:
#в столбце с количеством детей уберем значение 20, т.к. оно выглядит аномальным и заменим на медиану
data.loc[data['children'] == 20, ['children']] = data['children'].median()

In [17]:
# произведем замену количества детей с отрицательного на положительное
data['children'] = abs(data['children']).astype('int')

#и сразу проверим
data['children'].value_counts()

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [18]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Все замены прошли успешно.

### Обработка дубликатов

In [19]:
#проверим количество дубликатов в датасете
print ('Количество дубликатов в датасете:', data.duplicated().sum())

Количество дубликатов в датасете: 71


In [20]:
#удалим дубликаты и обновим индексацию датасета
data = data.drop_duplicates().reset_index(drop=True)

In [21]:
#снова проверим количество дубликатов в датасете
print ('Количество дубликатов в датасете:', data.duplicated().sum())

Количество дубликатов в датасете: 0


**Вывод**

Дубликаты были удалены. Так как в датасете нет персонализации клиентов (нет id или каких-либо других данных), то нельзя точно сказать, были это дубликаты или совпадения. Но таких строк было всего 71, поэтому их удаление сильно не повлияет на результаты исследования.

Возможно дубликаты образовались при повторном вводе информации сотрудниками банка, либо их появление можно связать с ошибками при выгрузке.

### Лемматизация

In [22]:
#проверим значения в столбце 'purpose'
data['purpose'].value_counts().sort_index()

автомобили                                478
автомобиль                                494
высшее образование                        452
дополнительное образование                460
жилье                                     646
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     471
на покупку подержанного автомобиля        478
на покупку своего автомобиля              505
на проведение свадьбы                     768
недвижимость                              633
образование                               447
операции с жильем                         652
операции с коммерческой недвижимостью     650
операции с недвижимостью                  675
операции со своей недвижимостью           627
покупка жилой недвижимости                606
покупка жилья                             646
покупка жилья для сдачи                   651
покупка жилья для семьи                   638
покупка коммерческой недвижимости 

В этом столбце можно классифицировать следующие категории: "недвижимость", "автомобиль", "образование" и "свадьба". Также можно добавить раздел "иные цели", если что-то не подойдет под одну из вышеуказанных категорий.

In [23]:
#для лемматизации используем функцию
def lemm(purpose):
    lemmas = m.lemmatize(purpose)
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    return 'иные цели'

In [24]:
#создадим новыц столбец, применив к столбцу 'purpose' лемматизацию
data['purpose_category'] = data['purpose'].apply(lemm)

In [25]:
#проверим, что в датасете появился новый столбец
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [26]:
#и проверим количество категорий в новом столбце 'purpose_category'
data['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

**Вывод**

При помощи лемматизации мы распределили цели кредита на несколько категорий, для удобства их последующих обработки и анализа. 

### Категоризация данных

Так как у нас в задании стоят вопросы о зависимости 4 различных факторов с возвратом кредита в срок, для ответа на этот вопрос нам нужно выделить 5 категорий:

1) Наличие детей;

2) Семейное положение;

3) Уровень дохода;

4) Цель кредита;

5) Наличие просроченной задолженности по кредиту.

In [27]:
#для начала проверим категории в столбце debt, т.к. он является основным для анализа
data['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

В столбце 'debt' всего 2 значения: 0 и 1, которые исходя из логики означают, что 0 - просроченной задолженности нет и 1 - просроченная задолженность есть.
Для удобства анализа и восприятия присвоим двум этим значениям категории "нет задолженности" и "просроченная задолженность.

In [28]:
#напишем функцию для присваивания категорий значениям 0 и 1 и при помощи метода apply создадим новый столбец с категориями
def debt_category(debt):
    if debt == 0:
        return 'нет задолженности'
    return 'просроченная задолженность'
data['debt_category'] = data['debt'].apply(debt_category)
data['debt_category'].value_counts()

нет задолженности             19713
просроченная задолженность     1741
Name: debt_category, dtype: int64

Следующим пунктом - наличие детей. Так как нас не интересует конкретное количество детей, мы выделим всего 2 категории: "есть дети" и "нет детей". Для этого тоже создадим функцию и новый столбец с категориями.

In [29]:
def children_category(children):
    if children == 0:
        return 'нет детей'
    return 'есть дети'
data['children_category'] = data['children'].apply(children_category)
data['children_category'].value_counts()

нет детей    14167
есть дети     7287
Name: children_category, dtype: int64

Семейное положение категоризировать не будем, т.к. данный столбец уже изначально содержит в себе всего 5 категорий.

In [30]:
#прежде чем выделять категории по доходу населения нужно определить его медиану
data['total_income'].median()

142594.0

По официальным данным от Росстата, средний доход населения России в 2020 году составил 35 361 рублей в месяц, среднемесячная зарплата — 51 083 рубля, средняя пенсия — 14 986 рублей. Так как у нас в исследовании присутствуют и пенсионеры, плюс не только Москва, а вся Россия, то будем брать за отправную точку среднемесячный доход в размере 35 361 рублей. Поэтому выделю 4 категории, учитывая медианный доход:
1) Менее 35 000 рублей - "низкий доход";

2) От 35 000 до 145 017 - "средний доход";

3) От 145 017 до 300 000 - "доход выше среднего";

4) От 300 000 - "высокий доход"

Как всегда напишем функцию и добавим новый столбец.

In [31]:
def income_category(total_income):
    if total_income < 35000:
        return 'низкий доход'
    elif total_income < 145017:
        return 'средний доход'
    elif total_income < 300000:
        return 'доход выше среднего'
    return 'высокий доход'
data['income_category'] = data['total_income'].apply(income_category)
data['income_category'].value_counts()

средний доход          11062
доход выше среднего     8840
высокий доход           1483
низкий доход              69
Name: income_category, dtype: int64

Категории по цели кредита были выделены при лемматизации данных.

Для удобства анализа данных создадим отдельную таблицу с созданными категориями и столбцом с семейным положением клиентов из основного датасета.

In [32]:
analysis = data[['children_category', 'family_status', 'income_category', 'purpose_category', 'debt_category']]
analysis.head()

,children_category,family_status,income_category,purpose_category,debt_category
0,есть дети,женат / замужем,доход выше среднего,недвижимость,нет задолженности
1,есть дети,женат / замужем,средний доход,автомобиль,нет задолженности
2,нет детей,женат / замужем,доход выше среднего,недвижимость,нет задолженности
3,есть дети,женат / замужем,доход выше среднего,образование,нет задолженности
4,нет детей,гражданский брак,доход выше среднего,свадьба,нет задолженности


**Вывод**

На этом шаге были выделены категории, необходимые для ответа на поставленные вопросы и добавлены в новую таблицу для удобства анализа.

## Шаг 3. Ответьте на вопросы

Для ответов на поставленные вопросы мы будем использовать метод сводных таблиц pivot_table()

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [33]:
#создаем сводную таблицу с подсчетом просроченной задолженности для категорий с детьми и без
children_pivot = analysis.pivot_table(index='children_category', columns='debt_category', values='family_status', aggfunc='count')

#добавим в таблицу children_pivot стоблец с процентным соотношением задолженности для большей наглядности
children_pivot['ratio'] = (children_pivot['просроченная задолженность']/children_pivot['нет задолженности'])*100

display(children_pivot)

debt_category,нет задолженности,просроченная задолженность,ratio
children_category,,,
есть дети,6617,670,10.125434
нет детей,13096,1071,8.178070


**Вывод**

Как мы видим из получившихся данных у заемщиков с детьми процент просрочки по кредитам выше аж на 2 % по сравнению с заемщиками без детьми, что можно связать с повышенной финансовой нагрузкой и обязательствами заемщиков с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
#теперь сделаем все то же самое для семейного положения
family_pivot = analysis.pivot_table(index='family_status', columns='debt_category', values='children_category', aggfunc='count')
family_pivot['ratio'] = (family_pivot['просроченная задолженность']/family_pivot['нет задолженности'])*100
display(family_pivot.sort_values('ratio'))

debt_category,нет задолженности,просроченная задолженность,ratio
family_status,,,
вдовец / вдова,896,63,7.031250
в разводе,1110,85,7.657658
женат / замужем,11408,931,8.160940
гражданский брак,3763,388,10.310922
не женат / не замужем,2536,274,10.804416


**Вывод**

Как мы видим из таблицы, люди, состоящие или ранее состоявшие в браке реже допускают просрочки, что можно связать с тем, что в семье несколько источников дохода, а также это, скорее всего, более взрослые и зрелые заемщики по сравнению с теми, кто никогда не состоял в браке или состоит в гражданском браке и поэтому они наиболее ответственно подходят к своим финансовым обязательствам.

Также из результатов данной таблицы можно сделать вывод, что разница в процентном соотношении между категориями "не женат / не замужем" и "гражданский брак" очень мала и они допускают больше всего просрочки.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
income_pivot = analysis.pivot_table(index='income_category', columns='debt_category', values='family_status', aggfunc='count')
income_pivot['ratio'] = (income_pivot['просроченная задолженность']/income_pivot['нет задолженности'])*100
display(income_pivot.sort_values('ratio'))

debt_category,нет задолженности,просроченная задолженность,ratio
income_category,,,
низкий доход,66,3,4.545455
высокий доход,1377,106,7.697894
доход выше среднего,8142,698,8.572832
средний доход,10128,934,9.221959


**Вывод**

Исходя из этих данных можно предположить, что просроченная задолженность возникает в основном у людей со средним доходом. 
У людей с низким доходом в данной таблице очень низкий процент просрочек, но это можно связать с малочисленностью данной категории заемщиков в текущем исследовании. Плюс в данную категорию попали в основном пенсионеры, а они очень ответственно относятся к своим обязательствам и стараются не допускать просрочек.

Можно сделать вывод, что у людей с высоким доходом бывает меньше всего просрочек по кредитам, если убрать статистическую погрешность данного исследования по людям с низким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [36]:
purpose_pivot = analysis.pivot_table(index='purpose_category', columns='debt_category', values='family_status', aggfunc='count')
purpose_pivot['ratio'] = (purpose_pivot['просроченная задолженность']/purpose_pivot['нет задолженности'])*100
display(purpose_pivot.sort_values('ratio'))

debt_category,нет задолженности,просроченная задолженность,ratio
purpose_category,,,
недвижимость,10029,782,7.797388
свадьба,2138,186,8.699719
образование,3643,370,10.156464
автомобиль,3903,403,10.325391


**Вывод**

Здесь мы видим, что меньше всего просрочек в кредитах, связанных с недвижимостью, а больше всего просрочек в автокредитовании и кредитах на образование.
Это можно соотнести с тем, что при кредитовании недвижимости кредит чаще всего является обеспеченным, то есть квартира находится в залоге у банка и при неоплате кредита банк может продать квартиру заемщика для погашения его задолженности перед банком. Также ипотечные кредиты чаще берут семейные пары, а как мы выяснили, они платят по кредитам лучше, чем заемщики, не состоящие в браке.

С кредитами на образование всё ясно - студент может не найти хорошую работу с достойной з/п после окончания учебы и возможности платить по кредиту у него не будет.

Автокредит хоть и является обеспеченным кредитом, но с автомобилями есть много рисков их утраты, а также дополнительных серьезных расходов на ремонт и обслуживание. В связи с этим, после серьезной аварии у клиента нет автомобиля (или он требует дорогостоящего ремонта) и заемщик перестает платить кредит, т.к. страха потерять предмет залога у него нет (а может от предмета залога ничего и не осталось).

## Шаг 4. Общий вывод

### Для ответов на поставленные вопросы представленные банком данные были предварительно обработаны и проанализированы. На этапе предварительной обработки было выявлено, что:
1. Наименования столбцов в датасете были правильные, переименовывать их не пришлось.
2. При этом в столбцах с количеством детей и количеством дней работы присутствовали значения со знаком "минус", чего быть не должно и данные значения были переведены в значения без знака минус;
3. В столбцах с образованием и семейным положением находились значения, записанные в разных регистрах и все они были приведены к нижнему регистру;
4. Пропуски были в 2174 строках, что составляет более 10 % значений и потеря этих данных могла сильно сказаться на результатах исследования, поэтому пропущенные значения в столбце с трудовым стажем были заменены нулями, так как эти данные не требуются для анализа, а вот пропуски по уровню дохода были заполнены медианными значениями в зависимости от типа занятости клиента (предприниматель, студент, сотрудник и т.д.);
5. В столбце с количеством детей помимо значений со знаком минус, находились 76 строк с количеством детей 20. Так как значений количества детей от 6 до 19 в датасете не присутствовало, был сделан вывод, что это ошибочные данные и произведена их замена медианным значением;
6. В таблице был обнаружен 71 дубликат строк. Так как в датасете нет персонализации клиентов (нет id или каких-либо других данных), то нельзя точно сказать, были это дубликаты или совпадения. Но таких строк было всего 71, поэтому их удаление не повлияет на результаты исследования;
7. Столбец с целями кредита для удобства анализа был разделен на следующие категории: "недвижимость", "автомобиль", "образование" и "свадьба", а также "иные цели", если что-то не подойдет под другие категории. Лемматизация целей проводилась с использованием библиотеки pymystem3.
8. Так как у нас в задании стоят вопросы о зависимости 4 различных факторов с фактом возврата кредита в срок, для ответа на этот вопрос из всего датасета были выделены 5 основных категорий для анализа:
- Наличие детей;
- Семейное положение;
- Уровень дохода;
- Цель кредита;
- Наличие просроченной задолженности по кредиту.

### Цель исследования была выяснить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

<b>1)</b> Исходя из анализа данных, можно сделать вывод о том, что <b>клиенты, состоящие в браке, являются более благонадежными заемщиками</b>, хотя разница в количестве просрочек с людьми, не состоящими в браке составляет всего 0.6 %, что является очень маленьким показателем.

<b>2)</b> А вот <b>наличие детей более негативно влияет на возможность заемщика вернуть кредит вовремя</b>. Разница в просрочке между заемщиками с детьми и без них составляет аж 1.99 %, что по банковским меркам довольно много.

<b>3)</b> Если смотреть на цели кредитования, то <b>меньше всего просрочек в кредитах, связанных с недвижимостью, а больше всего просрочек в автокредитовании и кредитах на образование</b>.
    
Это можно соотнести с тем, что при кредитовании недвижимости кредит чаще всего является обеспеченным, то есть квартира находится в залоге у банка и при неоплате кредита банк может продать квартиру заемщика для погашения его задолженности перед банком. Также ипотечные кредиты чаще берут семейные пары, а как мы выяснили, они платят по кредитам лучше, чем заемщики, не состоящие в браке.
    
С кредитами на образование всё ясно - студент может не найти хорошую работу с достойной з/п после окончания учебы и возможности платить по кредиту у него не будет.
    
Автокредит хоть и является обеспеченным кредитом, но с автомобилями есть много рисков их утраты, а также дополнительных серьезных расходов на ремонт и обслуживание. В связи с этим, после серьезной аварии у клиента нет автомобиля (или он требует дорогостоящего ремонта) и заемщик перестает платить кредит, т.к. страха потерять предмет залога у него нет (а может от предмета залога ничего и не осталось).
    
<b>4)</b> Исходя из проанализированных данных можно составить <b>портрет наиболее благонадежного заемщика: это человек, состоящий или состоявший в браке, пока еще не имеющий детей, берущий кредит на цели, связанные с недвижимостью и с доходом выше среднего или высоким доходом</b>.